# Importing libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pickle as pkl
import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Loading the data

In [2]:
with open('eng_fren/EN_FR.txt','r') as f:
  data = f.read()

len(data)

10506509

# Cleaning and preprocessing the data

In [3]:
# we need to clean the data
uncleaned_data_list = data.split('\n')

In [4]:
len(uncleaned_data_list)

152585

In [5]:
uncleaned_data_list = uncleaned_data_list[:152585]

In [6]:
len(uncleaned_data_list)

152585

In [7]:
english_word = []
french_word = []
# cleaned_data_list = []
for word in uncleaned_data_list:
    english_word.append(word.split('\t')[:-1][0])
    french_word.append(word.split('\t')[1]) 

IndexError: list index out of range

In [8]:
len(english_word), len(french_word)

(152583, 152583)

# Creating the dataframe and saving the data into csv

In [9]:
language_data = pd.DataFrame(columns=['English','French'])
language_data['English'] = english_word
language_data['French'] = french_word

In [10]:
# saving to csv
language_data.to_csv('language_data.csv', index=False)

In [11]:
language_data = pd.read_csv('language_data.csv')

In [12]:
language_data.head()

English      French
0     Go.        Va !
1    Run!     Cours !
2    Run!    Courez !
3    Wow!  Ça alors !
4   Fire!    Au feu !

In [13]:
language_data.tail()

English  \
152578  A carbon footprint is the amount of carbon dio...   
152579  Death is something that we're often discourage...   
152580  Since there are usually multiple websites on a...   
152581  If someone who doesn't know your background sa...   
152582  It may be impossible to get a completely error...   

                                                   French  
152578  Une empreinte carbone est la somme de pollutio...  
152579  La mort est une chose qu'on nous décourage sou...  
152580  Puisqu'il y a de multiples sites web sur chaqu...  
152581  Si quelqu'un qui ne connaît pas vos antécédent...  
152582  Il est peut-être impossible d'obtenir un Corpu...

## Data cleaning 

In [14]:
english_text = language_data['English'].values
french_text = language_data['French'].values

In [15]:
english_text[0], french_text[0]

('Go.', 'Va !')

In [16]:
len(english_text), len(french_text)

(152583, 152583)

In [17]:
english_text_ = [x.lower() for x in english_text]
french_text_ = [x.lower() for x in french_text]

In [18]:
type(english_text_), type(french_text_)

(list, list)

## Text Preprocessing

In [19]:
english_text_ = [re.sub("'",'',x) for x in english_text_]
french_text_ = [re.sub("'",'',x) for x in french_text_]
print(english_text_[0:3])

['go.', 'run!', 'run!']


In [20]:
def remove_punc(text_list):
  table = str.maketrans('', '', string.punctuation)
  removed_punc_text = []
  for sent in text_list:
    sentance = [w.translate(table) for w in sent.split(' ')]
    removed_punc_text.append(' '.join(sentance))
  return removed_punc_text
english_text_ = remove_punc(english_text_)
french_text_ = remove_punc(french_text_)

In [21]:
# removing the digits from english sentances
remove_digits = str.maketrans('', '', digits)
removed_digits_text = []
for sent in english_text_:
  sentance = [w.translate(remove_digits) for w in sent.split(' ')]
  removed_digits_text.append(' '.join(sentance))
english_text_ = removed_digits_text

# removing the digits from the french sentances
removed_digits_text = []
for sent in french_text_:
  sentance = [w.translate(remove_digits) for w in sent.split(' ')]
  removed_digits_text.append(' '.join(sentance))
french_text_ = removed_digits_text

# removing the stating and ending whitespaces
english_text_ = [x.strip() for x in english_text_]
french_text_ = [x.strip() for x in french_text_]

In [22]:
# Putting the start and end words in the french sentances
french_text_ = ["start " + x + " end" for x in french_text_]

In [23]:
# manipulated_french_text_
french_text_[0], english_text_[0]

('start va end', 'go')

In [24]:
len(french_text_),len(english_text)

(152583, 152583)

# Data spliting

In [25]:
X = english_text_
Y = french_text_

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.1)
len(X_train),len(y_train), len(X_test), len(y_test)

(137324, 137324, 15259, 15259)

In [27]:
X[0], Y[0]

('go', 'start va end')

In [28]:
X_train[0], y_train[0]

('several houses were washed away by the flood',
 'start de plusieurs maisons furent emportées par la crue end')

## Data preparing for encoder and decoder

In [29]:
# preparing data for the word embedding
def Max_length(data):
  max_length_ = max([len(x.split(' ')) for x in data])
  return max_length_

#Training data
max_length_english = Max_length(X_train)
max_lenght_french = Max_length(y_train)-1

#Test data
max_length_english_test = Max_length(X_test)
max_lenght_french_test = Max_length(y_test)+1

In [30]:
print(max_lenght_french, max_length_english)

print(max_lenght_french_test, max_length_english_test)

55 47
32 25


In [31]:
def tokenizer_(text_data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(text_data)
  return tokenizer

tokenizer_input = tokenizer_(X_train)
vocab_size_input = len(tokenizer_input.word_index) + 1
tokenizer_target = tokenizer_(y_train)
vocab_size_target = len(tokenizer_target.word_index) + 1

In [32]:
with open('tokenizer_input.pkl','wb') as f:
  pkl.dump(tokenizer_input,f)

with open('tokenizer_target.pkl','wb') as f:
  pkl.dump(tokenizer_target,f)
pkl.dump(tokenizer_input, open('tokenizer_input.pkl', 'wb'))
pkl.dump(tokenizer_target, open('tokenizer_target.pkl', 'wb'))

In [33]:
vocab_size_input,vocab_size_target

(13438, 27197)

In [34]:
def generator_batch(X= X_train,Y=y_train, batch_size=128):
  while True:
    for j in range(0, len(X), batch_size):
      encoder_data_input = np.zeros((batch_size,max_length_english),dtype='float32') #metrix of batch_size*max_length_english
      decoder_data_input = np.zeros((batch_size,max_lenght_french),dtype='float32') #metrix of batch_size*max_length_french
      decoder_target_input = np.zeros((batch_size,max_lenght_french,vocab_size_target),dtype='float32') # 3d array one hot encoder decoder target data
      for i, (input_text,target_text) in enumerate(zip(X[j:j+batch_size],Y[j:j+batch_size])):
        for t, word in enumerate(input_text.split()):
          encoder_data_input[i,t] = tokenizer_input.word_index[word] # Here we are storing the encoder 
                                                                     #seq in row here padding is done automaticaly as 
                                                                     #we have defined col as max_lenght
        for t, word in enumerate(target_text.split()):
          # if word == 'START_':
          #   word = 'start'
          # elif word == 'END_':
          #   word = 'end'
          decoder_data_input[i,t] = tokenizer_target.word_index[word] # same for the decoder sequence
          if t>0:
            decoder_target_input[i,t-1,tokenizer_target.word_index[word]] = 1 #target is one timestep ahead of decoder input because it does not have 'start tag'
      # print(encoder_data_input.shape())
      yield ([encoder_data_input,decoder_data_input],decoder_target_input)


# Model

In [35]:
latent_dim = 50
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,),name="encoder_inputs")
emb_layer_encoder = Embedding(vocab_size_input,latent_dim, mask_zero=True)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(emb_layer_encoder)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,),name="decoder_inputs")
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
emb_layer_decoder = Embedding(vocab_size_target,latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(emb_layer_decoder, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_target, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [36]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [37]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [38]:
model.fit_generator(generator = generator_batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size, epochs=epochs)

/home/rtm/tf/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
  44/1072 [>.............................] - ETA: 1:25:01 - loss: 1.1753 - accuracy: 0.1148

UnknownError:  KeyError: 'waou'
Traceback (most recent call last):

  File "/home/rtm/tf/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 249, in __call__
    ret = func(*args)

  File "/home/rtm/tf/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "/home/rtm/tf/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 892, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/rtm/tf/lib/python3.8/site-packages/keras/engine/data_adapter.py", line 822, in wrapped_generator
    for data in generator_fn():

  File "<ipython-input-34-3590fd4cdd49>", line 17, in generator_batch
    decoder_data_input[i,t] = tokenizer_target.word_index[word] # same for the decoder sequence

KeyError: 'waou'


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_14173]

Function call stack:
train_function


## Saving the model into Json

In [324]:
model_json = model.to_json()
with open("model_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_weight_5.h5")
print("Saved model to disk")

Saved model to disk


## Loading the model and weight from Json

In [325]:

# loading the model architecture and asigning the weights
json_file = open('model_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json)
# load weights into new model
model_loaded.load_weights("model_weight_5.h5")

# Inference

In [326]:
latent_dim = 50
#inference encoder
encoder_inputs_inf = model_loaded.input[0] #Trained encoder input layer
encoder_outputs_inf, inf_state_h, inf_state_c = model_loaded.layers[4].output # retoring the encoder lstm output and states
encoder_inf_states = [inf_state_h,inf_state_c]
encoder_model = Model(encoder_inputs_inf,encoder_inf_states)

In [327]:
#inference decoder
# The following tensor will store the state of the previous timestep in the "starting the encoder final time step"
decoder_state_h_input = Input(shape=(latent_dim,)) #becase during training we have set the lstm unit to be of 50
decoder_state_c_input = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_h_input,decoder_state_c_input]

# # inference decoder input
decoder_input_inf = model_loaded.input[1] #Trained decoder input layer
# decoder_input_inf._name='decoder_input'
decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
decoder_lstm_inf = model_loaded.layers[5]
decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state =decoder_state_input)
decoder_state_inf = [decoder_state_h_inf,decoder_state_c_inf]
#inference dense layer
dense_inf = model_loaded.layers[6]
decoder_output_final = dense_inf(decoder_output_inf)# A dense softmax layer to generate prob dist. over the target vocabulary

decoder_model = Model([decoder_input_inf]+decoder_state_input,[decoder_output_final]+decoder_state_inf)



## loading the saved tokenizer for the prediction

In [328]:
with open('tokenizer_input.pkl','rb') as f:
  tokenizer_input = pkl.load(f)
with open('tokenizer_target.pkl','rb') as f:
  tokenizer_target = pkl.load(f)
# Creating the reverse mapping to get the word from the index in the sequence
reverse_word_map_input = dict(map(reversed, tokenizer_input.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_target.word_index.items()))

## Code for generating the translated sentance

In [329]:
# Code to predct the input sentences translation
def decode_seq(input_seq):
  # print("input_seq=>",input_seq)
  state_values_encoder = encoder_model.predict(input_seq)
  # intialize the target seq with start tag
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tokenizer_target.word_index['start']
  # print("target_seq:=>",target_seq)
  stop_condition = False
  decoder_sentance = ''
  # print("Beforee the while loop")
  while not stop_condition:
    sample_word , decoder_h,decoder_c= decoder_model.predict([target_seq] + state_values_encoder)
    # print("sample_word: =>",sample_word)
    sample_word_index = np.argmax(sample_word[0,-1,:])
    # print("sample_word_index: ",sample_word_index)
    decoder_word = reverse_word_map_target[sample_word_index]
    decoder_sentance += ' '+ decoder_word
    # print("decoded word:=>",decoder_word)
    # print(len(decoder_sentance))
    # print("len(decoder_sentance) > 70: ",len(decoder_sentance) > 70)
    # print('decoder_word == "end"',decoder_word == 'end')
    # print(decoder_word == 'end' or len(decoder_sentance) > 70)
    # stop condition for the while loop
    if (decoder_word == 'end' or 
        len(decoder_sentance) > 70):
        stop_condition = True
        # print("from if condition")
    # target_seq = np.zeros((1,1))
    target_seq[0, 0] = sample_word_index
    # print(target_seq)
    state_values_encoder = [decoder_h,decoder_c]
  return decoder_sentance


## some sentance translation from english sentance

In [330]:
for i in range(10):
  sentance = X_test[i]
  original_target = y_test[i]
  input_seq = tokenizer_input.texts_to_sequences([sentance])
  pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
  # print('input_sequence =>',input_seq)
  # print("pad_seq=>",pad_sequence)
  predicted_target = decode_seq(pad_sequence)
  print("Test sentance: ",i+1)
  print("sentance: ",sentance)
  print("origianl translate:",original_target[6:-4])
  print("predicted Translate:",predicted_target[:-4])
  print("=="*50)


Test sentance:  1
sentance:  quand vous êtes venu la cérémonie avait lieu
origianl translate: وصلت أثناء الحفلة
predicted Translate: 
Test sentance:  2
sentance:  nous ne comprenons vraiment pas les femmes
origianl translate: نحن لا نفهم النساء حقا
predicted Translate: 
Test sentance:  3
sentance:  puisje payer par carte de crédit
origianl translate: أيمكنني الدفع عن طريق بطاقة الإئتمان؟
predicted Translate: 
Test sentance:  4
sentance:  désolé pour le dérangement
origianl translate: المعذره على الإزعاج
predicted Translate: 
Test sentance:  5
sentance:  puisje parler au médecin
origianl translate: أيمكنني أن أكلم الطبيب؟
predicted Translate: 
Test sentance:  6
sentance:  sami met sa voiture dans le garage
origianl translate: وضع سامي سيّارته في المرآب
predicted Translate: 
Test sentance:  7
sentance:  nous navons rien à perdre
origianl translate: لا يوجد لدينا ما نخسره
predicted Translate: 
Test sentance:  8
sentance:  kate est plus maligne que nimporte quel autre élève de notre classe

In [332]:
sentance = "i play a game"
input_seq = tokenizer_input.texts_to_sequences([sentance])
pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
# print('input_sequence =>',input_seq)
# print("pad_seq=>",pad_sequence)
predicted_target = decode_seq(pad_sequence)
print("predicted Translate:",predicted_target[:-4])

predicted Translate: 
